In [2]:
!git clone https://github.com/rcmalli/keras-vggface.git

Cloning into 'keras-vggface'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 281 (delta 12), reused 6 (delta 2), pack-reused 251
Receiving objects: 100% (281/281), 6.03 MiB | 9.99 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!unzip /content/drive/My\ Drive/dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/dfdc/validation/REAL/REAL_GOs7HeBnReETmrJJ.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_goVqQPXYAXUlaz6n.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_gPBOBGHH1w0iA0Zt.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_GPcLZq5ZO0greYYI.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_gPdSRCpV80d1lvTm.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_GPmYgZTYVlrGD3Av.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_GpQGcmfipWa95THT.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_gpqjqCFkAvlP1HhF.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_GPUzk3i0ZXj84Q6y.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_gPVFdT1kjjutGfef.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_GPWQTV3UBr9mqrfx.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_gpZIfgjwSaR1eIfv.jpg  
  inflating: dataset/dfdc/validation/REAL/REAL_GqNiiJIQNaT4xWwU.jpg  
  inflating: dataset/dfdc/validation/RE

In [5]:
import os
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
from keras.preprocessing import image
from kerasvggface.keras_vggface.vggface import VGGFace
from kerasvggface.keras_vggface import utils
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import config
from imutils import paths
from keras.layers.core import Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

# derive the paths to the training, validation, and testing
# directories
trainPath = os.path.sep.join([config.BASE_PATH, config.TRAIN])
valPath = os.path.sep.join([config.BASE_PATH, config.VAL])
testPath = os.path.sep.join([config.BASE_PATH, config.TEST])

# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=False,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# initialize the training generator
trainGen = trainAug.flow_from_directory(
	trainPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=config.BATCH_SIZE)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	valPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BATCH_SIZE)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	testPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BATCH_SIZE)

labels = (trainGen.class_indices)
print("..." , labels)

def my_gen(gen):
  i = 0
  while True:
    try:
        data, labels = next(gen)
        yield data, labels
    except:
        print(i)
        i += 1
        pass

#custom parameters
nb_class = 2
hidden_dim = 512

resnet_model = VGGFace( model='resnet50' , include_top=False, input_shape=(224, 224, 3))
resnet_model.summary()

print('Len layers: ' , len(resnet_model.layers))

last_layer = resnet_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fully_connected_1')(x)
x = Dropout(0.5)(x)
out = Dense(nb_class, activation='softmax', name='fully_connected_2')(x)

custom_resnet_model = Model(resnet_model.input, out)

#loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in resnet_model.layers:
	layer.trainable = False

# checkpoint
filepath= "drive/My Drive/VGGFace_ResNet50_headmodel/weights.{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
callbacks_list = [checkpoint]

# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
custom_resnet_model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = custom_resnet_model.fit_generator(
	my_gen(trainGen),
	steps_per_epoch=totalTrain // config.BATCH_SIZE,
	validation_data=my_gen(valGen),
	validation_steps=totalVal // config.BATCH_SIZE,
	epochs=15 , callbacks = callbacks_list)

# PLOT
history = H
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Found 149553 images belonging to 2 classes.
Found 18691 images belonging to 2 classes.
Found 18705 images belonging to 2 classes.
... {'FAKE': 0, 'REAL': 1}











KeyboardInterrupt: ignored

In [0]:
!pip install tensorflow==1.14.

     |████████████████████████████████| 109.2MB 92kB/s 
     |████████████████████████████████| 3.2MB 46.5MB/s 
     |████████████████████████████████| 491kB 50.4MB/s 
  Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc1
    Uninstalling tensorflow-2.2.0rc1:
      Successfully uninstalled tensorflow-2.2.0rc1


In [0]:
import config
from keras.models import load_model
import os
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
from keras.preprocessing import image
from kerasvggface.keras_vggface.vggface import VGGFace
from kerasvggface.keras_vggface import utils
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import config
from imutils import paths
from keras.layers.core import Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

# derive the paths to the training, validation, and testing
# directories
trainPath = os.path.sep.join([config.BASE_PATH, config.TRAIN])
valPath = os.path.sep.join([config.BASE_PATH, config.VAL])
testPath = os.path.sep.join([config.BASE_PATH, config.TEST])

# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=False,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# initialize the training generator
trainGen = trainAug.flow_from_directory(
	trainPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=config.BATCH_SIZE)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	valPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BATCH_SIZE)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	testPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BATCH_SIZE)

labels = (trainGen.class_indices)
print("..." , labels)

def my_gen(gen):
  i = 0
  while True:
    try:
        data, labels = next(gen)
        yield data, labels
    except:
        print(i)
        i += 1
        pass

#custom parameters
nb_class = 2
hidden_dim = 512

resnet_model = VGGFace( model='resnet50' , include_top=False, input_shape=(224, 224, 3))
resnet_model.summary()

print('Len layers: ' , len(resnet_model.layers))

last_layer = resnet_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fully_connected_1')(x)
x = Dropout(0.5)(x)
out = Dense(nb_class, activation='softmax', name='fully_connected_2')(x)

custom_resnet_model = Model(resnet_model.input, out)

# reset the testing generator and evaluate the network after
# fine-tuning just the network head
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()

# reset our data generators
trainGen.reset()
valGen.reset()

#loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in resnet_model.layers:
	layer.trainable = False

# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in resnet_model.layers[0:]:
	layer.trainable = True

# loop over the layers in the model and show which ones are trainable
# or not
for layer in resnet_model.layers:
	print("{}: {}".format(layer, layer.trainable))

custom_resnet_model.load_weights("drive/My Drive/VGGFace_ResNet50_headmodel/weights.15-0.45.hdf5")

# checkpoint
filepath= "drive/My Drive/VGGFace_ResNet50_custommodel/weights.{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
callbacks_list = [checkpoint]

# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
# opt = SGD(lr=1e-4, momentum=0.9)
custom_resnet_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False),metrics=['accuracy'])

# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
H = custom_resnet_model.fit_generator(
	my_gen(trainGen),
	steps_per_epoch=totalTrain // config.BATCH_SIZE,
	validation_data=my_gen(valGen),
	validation_steps=totalVal // config.BATCH_SIZE,
	epochs=15 , callbacks=callbacks_list)

# serialize the model to disk
print("[INFO] serializing network...")
custom_resnet_model.save(config.MODEL_PATH)

# PLOT
history = H
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

ERROR! Session/line number was not unique in database. History logging moved to new session 60
Found 149553 images belonging to 2 classes.
Found 18691 images belonging to 2 classes.
Found 18705 images belonging to 2 classes.
... {'FAKE': 0, 'REAL': 1}
0


Exception ignored in: <generator object my_gen at 0x7fc970fbefc0>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object my_gen at 0x7fc970759360>
RuntimeError: generator ignored GeneratorExit


0
Model: "vggface_resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_4[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_99 (Activation)      (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
_________________________________________________________________________________